Name: Wenhao Wu

ID: 1304952

In [44]:
#ref https://livebook.manning.com/#!/book/deep-learning-with-python/chapter-3/316

import numpy as np
import keras
import matplotlib.pyplot as plt

In [45]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

mean = np.mean(train_data, axis=0)
std = np.std(train_targets, axis=0)

train_data -= mean
train_data /= (2*std)

test_data -= mean
test_data /= (2*std)

The above cell is to normalize the input, we should somehow shuffle the data first.

In [46]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

We define a method to init the model, since in k-fold validation we need to init a clean model multiple times

In [47]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #',i)
    
    begin = i * num_val_samples
    end = (i+1) * num_val_samples
 
    vd = train_data[begin: end]
    vt = train_targets[begin: end]
        
    ptd = np.concatenate([train_data[:begin], train_data[end:]], axis=0)
    ptt = np.concatenate([train_targets[:begin], train_targets[end:]], axis=0)
    
    model = build_model()
    model.fit(ptd, ptt, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(vd, vt, verbose=0)
    all_scores.append(val_mae)

print("Training Done")

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
Training Done


In [48]:
print(np.mean(all_scores))

2.4990354192138895


In [49]:
model_normal = build_model()
hist = model_normal.fit(train_data, train_targets, epochs=100, batch_size = 1, verbose=0)

val_mse, val_mae = model_normal.evaluate(test_data, test_targets, verbose=0)

print(val_mae)


2.9497352768393124


The above cell uses a normal approach to train the model and the mean absolute error raises to **2.94** compares to **2.49** in the k-fold approach.   